# CO70058 Computer Vision
# Tutorial 1 - Edge Detection and Pre-processing



# Convolution

# Why do we need to flip the mask?

Because otherwise we would be doing a cross-correlation instead of a convolution.



Watch this video:
[Convolution vs Cross Correlation](https://www.youtube.com/watch?v=C3EEy8adxvc). For a 3x3 kernel, **k**=1. **h** is the kernel, and the coordinate 0,0 of the kernel is the pixel in the centre. **F** is input image and **G** is the resulting image after the convolution.

If you still have questions let me know and I will add more detail here!

# Lab

We will be using OpenCV and Numpy so let's first import these packages:

In [ ]:
import cv2
import numpy as np

## Question 1

First we need to create the image:

In [ ]:
img_row = np.array([3, 4, 8, 15, 25, 44, 50, 52], dtype=np.uint8)
img_1 = np.tile(img_row, (8, 1))
print(img_1)

[[ 3  4  8 15 25 44 50 52]
 [ 3  4  8 15 25 44 50 52]
 [ 3  4  8 15 25 44 50 52]
 [ 3  4  8 15 25 44 50 52]
 [ 3  4  8 15 25 44 50 52]
 [ 3  4  8 15 25 44 50 52]
 [ 3  4  8 15 25 44 50 52]
 [ 3  4  8 15 25 44 50 52]]


### 1.A - Prewitt operator

To compute the convolution we will use the [filter2D](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga27c049795ce870216ddfb366086b5a04) function of OpenCV. If you open that link you will see them saying that that function computes correlation, not the convolution! That is, the kernel is not mirrored around the anchor point. If you need a real convolution, flip the kernel using flip and set the new anchor to `(kernel.cols - anchor.x - 1, kernel.rows - anchor.y - 1)`.

In our case, given the 3x3 kernels, we can assume that the anchor of the kernel is in the middle of the image and therefore the new anchor will remain in the same position. So all we need to do is to flip the kernel twice, once relative to x and once relative to y.

Let's start by creating the kernels:

In [ ]:
prewitt_x = np.array([[-1, 0, 1],
                      [-1, 0, 1],
                      [-1, 0, 1]], dtype=np.float32)
prewitt_y = np.array([[ 1, 1, 1],
                      [ 0, 0, 0],
                      [-1,-1,-1]], dtype=np.float32)

Now let's flip them using [np.flip()](https://numpy.org/doc/stable/reference/generated/numpy.flip.html):

In [ ]:
prewitt_x_flip = np.flip(prewitt_x, 1)
prewitt_x_flip = np.flip(prewitt_x_flip, 0)
print('prewitt_x_flip:\n{}'.format(prewitt_x_flip))

prewitt_y_flip = np.flip(prewitt_y, 1)
prewitt_y_flip = np.flip(prewitt_y_flip, 0)
print('prewitt_y_flip:\n{}'.format(prewitt_y_flip))

prewitt_x_flip:
[[ 1.  0. -1.]
 [ 1.  0. -1.]
 [ 1.  0. -1.]]
prewitt_y_flip:
[[-1. -1. -1.]
 [ 0.  0.  0.]
 [ 1.  1.  1.]]


Now let's do the convolution. We add the absolute values together so that we don't cancel the effect of the different gradients.

In [ ]:
img_prewitt_x = cv2.filter2D(img_1, cv2.CV_64F, prewitt_x_flip)
img_prewitt_y = cv2.filter2D(img_1, cv2.CV_64F, prewitt_y_flip)
img_prewitt = abs(img_prewitt_x) + abs(img_prewitt_y)
print("\t1_A : \n{}".format(img_prewitt))

	1_A : 
[[ 0. 15. 33. 51. 87. 75. 24.  0.]
 [ 0. 15. 33. 51. 87. 75. 24.  0.]
 [ 0. 15. 33. 51. 87. 75. 24.  0.]
 [ 0. 15. 33. 51. 87. 75. 24.  0.]
 [ 0. 15. 33. 51. 87. 75. 24.  0.]
 [ 0. 15. 33. 51. 87. 75. 24.  0.]
 [ 0. 15. 33. 51. 87. 75. 24.  0.]
 [ 0. 15. 33. 51. 87. 75. 24.  0.]]


By default OpenCV uses as a border the BORDER_REFLECT_101, [see more here](https://docs.opencv.org/master/d2/de8/group__core__array.html#ga209f2f4869e304c82d07739337eae7c5).

When you are doing it by hand you can choose any type of border that you want, you could for example set all the border values to zero. So the values in the border will change according the border that you define.

Technical note: use `cv2.CV_64F` if you are unsure about which type to use in the second argument of the function. Essentialy here we want to avoid [missing edges](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_gradients/py_gradients.html#one-important-matter).

An alternative way to combine the gradients would be using [convertScaleAbs()](https://docs.opencv.org/3.4/d2/de8/group__core__array.html#ga3460e9c9f37b563ab9dd550c4d8c4e7d) and [addWeighted()](https://docs.opencv.org/3.4/d2/de8/group__core__array.html#gafafb2513349db3bcff51f54ee5592a19), as done in [this OpenCV tutorial](https://docs.opencv.org/3.4/d2/d2c/tutorial_sobel_derivatives.html):

In [ ]:
# Scales, calculates absolute values, and converts the result to 8-bit (CV_8U)
abs_grad_x = cv2.convertScaleAbs(img_prewitt_x)
abs_grad_y = cv2.convertScaleAbs(img_prewitt_y)
    
# Calculate the weighted sum of the two gradients
grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0) # Some people just prefer to sum directly (abs_grad_x + abs_grad_y)

### 1.B - Sobel

Let's do the same but for the sobel now:

In [ ]:
sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]], dtype=np.float32)
sobel_y = np.array([[ 1, 2, 1],
                    [ 0, 0, 0],
                    [-1,-2,-1]], dtype=np.float32)

Let's flip:

In [ ]:
sobel_x_flip = np.flip(sobel_x, 1)
sobel_x_flip = np.flip(sobel_x_flip, 0)
print('sobel_x_flip:\n{}'.format(sobel_x_flip))

sobel_y_flip = np.flip(sobel_y, 1)
sobel_y_flip = np.flip(sobel_y_flip, 0)
print('sobel_y_flip:\n{}'.format(sobel_y_flip))

sobel_x_flip:
[[ 1.  0. -1.]
 [ 2.  0. -2.]
 [ 1.  0. -1.]]
sobel_y_flip:
[[-1. -2. -1.]
 [ 0.  0.  0.]
 [ 1.  2.  1.]]


And the convolution:

In [ ]:
img_sobel_x = cv2.filter2D(img_1, cv2.CV_64F, sobel_x_flip)
img_sobel_y = cv2.filter2D(img_1, cv2.CV_64F, sobel_y_flip)
img_sobel = abs(img_sobel_x) + abs(img_sobel_y)
print("\t1_B : \n{}".format(img_sobel))

	1_B : 
[[  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]]


Now, you may be asking: "Couldn't we have used the OpenCV inbuilt function of sobel [cv2.Sobel()](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gacea54f142e81b6758cb6f375ce782c8d)?"

Well, let's try:

In [ ]:
img_sobel_x = cv2.Sobel(img_1, cv2.CV_64F, 1, 0, ksize=3)
img_sobel_y = cv2.Sobel(img_1, cv2.CV_64F, 0, 1, ksize=3)
img_sobel = abs(img_sobel_x) + abs(img_sobel_y)
print("\t1_B : \n{}".format(img_sobel))

	1_B : 
[[  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]
 [  0.  20.  44.  68. 116. 100.  32.   0.]]


We get the same!

### 1.C - Laplacian

Let's do the same:

In [ ]:
laplacian = np.array([[0,  1, 0],
                      [1, -4, 1],
                      [0,  1, 0]], dtype=np.float32)

In this case the kernel is symmetrical so we do not need to worry about flipping!

In [ ]:
img_laplacian = cv2.filter2D(img_1, cv2.CV_64F, laplacian)
print("\t1_C : \n{}".format(img_laplacian))

	1_C : 
[[  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]]


Now, let's do it with the inbuilt function of OpenCV [cv2.Laplacian()](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gad78703e4c8fe703d479c1860d76429e6)

In [ ]:
img_laplacian = cv2.Laplacian(img_1, cv2.CV_64F, ksize=1) # ksize=1 gives the expected kernel: https://docs.opencv.org/3.4/da/d85/tutorial_js_gradients.html
print("\t1_C : \n{}".format(img_laplacian))

	1_C : 
[[  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]
 [  2.   3.   3.   3.   9. -13.  -4.  -4.]]


We get the same!

Comment on 1:

From the Prewitt and Sobel Operator we can see that in the edge area, the pixel intensity shows a "jump" or a high variation of intensity. Getting the first derivative of the intensity, we observe that an edge is characterized by a maximum or a minimum. What happens if we take the second derivative? On the edge, the second derivative is zero! So, we can also use this criterion to attempt to detect edges in an image. However, note that zeros will not only appear in edges (they can actually appear in other meaningless locations); this can be solved by applying filtering where needed.
    
[Read more](https://docs.opencv.org/3.4/d5/db5/tutorial_laplace_operator.html)

## Question 2

First we need to create the image:

In [ ]:
img_2 = np.array([[7, 12, 9],
                  [6,  7, 8],
                  [3,  4, 5]], dtype=np.uint8)

Let's use the flipped kernels again to do the convolution:

In [ ]:
grad_x_a = cv2.filter2D(img_2, cv2.CV_64F, prewitt_x_flip)
grad_y_a = cv2.filter2D(img_2, cv2.CV_64F, prewitt_y_flip)
grad_x_a = abs(grad_x_a)
grad_y_a = abs(grad_y_a)

Now let's compute the magnitude and direction:

In [ ]:
# Get square root of sum of squares
magnitude_a = np.hypot(grad_x_a, grad_y_a)
print('magnitude:\n{}'.format(magnitude_a))
# Get angles in radians
grad_direction_a = np.arctan2(grad_y_a, grad_x_a)
# Convert radians to degrees
grad_direction_a = grad_direction_a * 180 / np.pi
print('grad_direction:\n{}'.format(grad_direction_a))
print("\t2_A:\n{} degrees".format(grad_direction_a[1, 1])) # Get pixel in the middle [1, 1]

magnitude:
[[ 0.          6.          0.        ]
 [20.         17.08800749 20.        ]
 [ 0.          6.          0.        ]]
grad_direction:
[[ 0.          0.          0.        ]
 [90.         69.44395478 90.        ]
 [ 0.          0.          0.        ]]
	2_A:
69.44395478041653 degrees


Now let's do the same for the Sobel kernel:

In [ ]:
grad_x_b = cv2.filter2D(img_2, cv2.CV_64F, sobel_x_flip)
grad_y_b = cv2.filter2D(img_2, cv2.CV_64F, sobel_y_flip)
grad_x_b = abs(grad_x_b)
grad_y_b = abs(grad_y_b)

Now let's compute the magnitude and direction:

In [ ]:
# Get square root of sum of squares
magnitude_b = np.hypot(grad_x_b, grad_y_b)
print('magnitude:\n{}'.format(magnitude_b))
# Get angles in radians
grad_direction_b = np.arctan2(grad_y_b, grad_x_b)
# Convert radians to degrees
grad_direction_b = grad_direction_b * 180 / np.pi
print('grad_direction:\n{}'.format(grad_direction_b))
print("\t2_B:\n{} degrees".format(grad_direction_b[1, 1])) # Get pixel in the middle [1, 1]

magnitude:
[[ 0.          8.          0.        ]
 [24.         25.29822128 24.        ]
 [ 0.          8.          0.        ]]
grad_direction:
[[ 0.          0.          0.        ]
 [90.         71.56505118 90.        ]
 [ 0.          0.          0.        ]]
	2_B:
71.56505117707799 degrees


Comment on 2:

    Both the results are very similar. Prewitt operator is similar to the Sobel operator and is used for detecting vertical and horizontal edges in images. However, unlike the Sobel, this operator does not place any emphasis on the pixels that are closer to the center of the mask.

## Question 3

Let's start by creating the kernel, notice that it is simmetric so no need to flip.

In [ ]:
gaussian = np.array([[1.0/36, 1.0/9, 1.0/36],
                     [1.0/9 , 4.0/9, 1.0/9 ],
                     [1.0/36, 1.0/9, 1.0/36]], dtype=np.float64) # float64 since we will be doing a SVD decomposition

In [ ]:
img_gaussian = cv2.filter2D(img_1, cv2.CV_64F, gaussian)
print("\tQ4: Filtered image:\n{}".format(img_gaussian))

	Q4: Filtered image:
[[ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]]


Another option would be to use the OpenCV inbuilt function [cv2.GaussianBlur()](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gaabe8c836e97159a9193fb0b11ac52cf1) to compute the Gaussian:

In [ ]:
img_gaussian = cv2.GaussianBlur(img_1, (3, 3), 0)
print(img_gaussian)

[[ 4  5  9 16 27 41 49 51]
 [ 4  5  9 16 27 41 49 51]
 [ 4  5  9 16 27 41 49 51]
 [ 4  5  9 16 27 41 49 51]
 [ 4  5  9 16 27 41 49 51]
 [ 4  5  9 16 27 41 49 51]
 [ 4  5  9 16 27 41 49 51]
 [ 4  5  9 16 27 41 49 51]]


Filtering an M-by-N image with a P-by-Q filter kernel requires roughly MNPQ multiplies and MN(PQ - 1) adds.
In our case with a 3x3 kernel, P=Q=3.

In [ ]:
img_h, img_w = img_1.shape
P = 3.0
Q = 3.0
n_multiplications = img_h * img_w * P * Q
n_additions = img_h * img_w * ((P * Q) - 1.0)
print("Computation cost:\n {} multiplications and {} additions".format(int(n_multiplications), int(n_additions)))

Computation cost:
 576 multiplications and 512 additions


Now how could we separate the 2D kernel into two 1D kernels?

Well, we can do that if it is a separable filter.
[What is a separable filter?](https://blogs.mathworks.com/steve/2006/10/04/separable-convolution/)

Basically, it is a two-dimensional filter kernel is separable if it can be expressed as the `outer product` of two vectors.

[How can I determine if a matrix is an outer product of two vectors? How to determine these vectors?](https://blogs.mathworks.com/steve/2006/11/28/separable-convolution-part-2/)

Well if it is separable we will find the two vectors using an SVD decomposition as mentioned in that website.
        

In [ ]:
u, s, vh  = np.linalg.svd(gaussian)

d_y = u[:,0] * np.sqrt(s[0]) # should give this result = [1/6, 2/3, 1/6]
print("\tQ4: Calculated d_y:\n{}".format(d_y))

d_x = np.transpose(vh)[:,0] * np.sqrt(s[0]) # should give this result = [1/6, 2/3, 1/6]
print("\tQ4: Calculated d_x:\n{}".format(d_x))

	Q4: Calculated d_y:
[-0.16666667 -0.66666667 -0.16666667]
	Q4: Calculated d_x:
[-0.16666667 -0.66666667 -0.16666667]


If these are indeed the 1D kernels then their outer product should give the same as the gaussian kernel.
Let's try that:

In [ ]:
print("Should be zero:\n{}".format(gaussian - np.outer(d_y, d_x)))

Should be zero:
[[ 1.38777878e-17  9.71445147e-17  2.08166817e-17]
 [-2.77555756e-17  0.00000000e+00 -1.38777878e-17]
 [-6.93889390e-18  0.00000000e+00 -3.46944695e-18]]


Which is aprox. 0 as expected!


---





Let's give it a try with the 1D convolutions
   - Here we can use [np.convolve](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html) to do 1D convolutions

To get the same result as OpenCV first we need to add the same border around it
   - since we have a 3x3 kernel we need to add a border of size two.
   - also by default OpenCV used BORDER_REFLECT_101 when doing convolutions, also let's do the same here

In [ ]:
BORDER_SIDE = 1
img_4 = cv2.copyMakeBorder(img_1, BORDER_SIDE, BORDER_SIDE, BORDER_SIDE, BORDER_SIDE, borderType=cv2.BORDER_REFLECT_101)
print("Image with border:\n{}".format(img_4))

Image with border:
[[ 4  3  4  8 15 25 44 50 52 50]
 [ 4  3  4  8 15 25 44 50 52 50]
 [ 4  3  4  8 15 25 44 50 52 50]
 [ 4  3  4  8 15 25 44 50 52 50]
 [ 4  3  4  8 15 25 44 50 52 50]
 [ 4  3  4  8 15 25 44 50 52 50]
 [ 4  3  4  8 15 25 44 50 52 50]
 [ 4  3  4  8 15 25 44 50 52 50]
 [ 4  3  4  8 15 25 44 50 52 50]
 [ 4  3  4  8 15 25 44 50 52 50]]


In [ ]:
img_h_with_broder, img_w_with_broder = img_4.shape
result = np.zeros((img_h_with_broder, img_w_with_broder))

for v in range(img_h_with_broder):
    result[v] = np.convolve(img_4[v,:], d_x, 'same')
for u in range(img_w_with_broder):
    result[:,u] = np.convolve(result[:,u], d_y, 'same')

# Finally let's crop again to get the initial size
result = result[BORDER_SIDE:BORDER_SIDE+img_h, BORDER_SIDE:BORDER_SIDE+img_w]
print("\tQ4: Filtered image using 2x 1D vectors:\n{}".format(result))

	Q4: Filtered image using 2x 1D vectors:
[[ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]
 [ 3.33333333  4.5         8.5        15.5        26.5        41.83333333
  49.33333333 51.33333333]]


We got the same result as before! Now let's look into the computational cost:

Theoretically, when filtering an M-by-N image with a P-by-Q filter kernel, if the kernel is separable, you can filter in two steps.
The first step requires about MNP multiplies and MN(P-1) adds. The second requires about MNQ multiplies and MN(Q-1) adds, for a total of MN(P + Q) multiplies and MN(P + Q - 2) adds.

In our case with a 3x3 kernel, P=Q=3

In [ ]:
n_multiplications = img_h * img_w * (P + Q)
n_additions = img_h * img_w * (P + Q - 2.0)
print("Computation cost:\n {} multiplications and {} additions".format(int(n_multiplications), int(n_additions)))

Computation cost:
 384 multiplications and 256 additions


As we see it's better than before. However, in practice we should take into consideration that OpenCV is highly optimized and for sufficiently large kernels (~11 x 11 or larger) it uses the discrete Fourier Transform to speed up the process.

## Question 4

You will learn in the next tutorials more about 3D reconstruction. For 3D reconstruction most algorithms match regions (such as corners and edges) between multiple pictures of the same scene. However, in the presence of reflections, the algorithm may get confused since it could match the features in the reflection with features in the physical objects.

With the coming tutorials I think you will understand what I am talking about here, but if you don't please just ask!